In [0]:
%sql
CREATE OR REPLACE TABLE dados_clientes.gold.clientes_contatos_validos AS
SELECT 
  nome,
  cpf,
  dataNascimento,
  cargo,
  empresa,
  email,
  estado,
  endereco,
  regexp_replace(telefone, "[+ ( ) -]", "") as telefone_limpo,
  CASE 
    WHEN regexp_replace(telefone, "[+ ( ) -]", "") RLIKE '^[1-9][1-9]9[0-9]{8}$' THEN true 
    ELSE false 
  END AS is_contato_valido,
  website,
  dataReferenciaDado,
  current_timestamp() as dataUltimaAtualizacao
FROM dados_clientes.silver.dados_clientes_silver 
WHERE substr(regexp_replace(telefone, "[+ ( ) -]", ""), 1, 4) NOT IN ('0300', '0500', '0800', '0900')
QUALIFY ROW_NUMBER() OVER (PARTITION BY cpf ORDER BY dataReferenciaDado DESC) = 1;